In [1]:
%pylab inline
from IPython.display import HTML
from __future__ import print_function
from decimal import Decimal
import urllib2
import json
import sc
import time, random

Populating the interactive namespace from numpy and matplotlib


In [2]:
HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
prefix = 'http://api.worldbank.org'

countries = '/countries/cn;ru;us'

times = '?date=1999:2014'

parameters = '/indicators/MS.MIL.XPND.GD.ZS'

limit = '?per_page=48'

jsonKey = '&format=json'

f = urllib2.urlopen(prefix + countries + parameters + times + jsonKey)
queryData = f.read()

In [74]:
jsonData = json.loads(queryData)

In [75]:
cnData = []
ruData = []
usData = []
for i in jsonData[1]:
    valueString = str(i['value'])
    if valueString == 'None':
        thisValue = 0
    else:
        thisValue = float(str(i['value']))
    if i['country']['value'] == 'China':
        cnData.append(thisValue)
    elif i['country']['value'] == 'Russian Federation':
        ruData.append(thisValue)
    elif i['country']['value'] == 'United States':
        usData.append(thisValue)

In [76]:
maxVal = max( max(usData), max(ruData), max(cnData))
minVal = min( min(usData), min(ruData), min(cnData))

In [77]:
minFreq = 200
maxFreq = 2000

cnFreqs = []
ruFreqs = []
usFreqs = []

cnAmps = []
ruAmps = []
usAmps = []

for i in cnData:
    cnFreqs.append(((i - minVal) / (maxVal - minVal)) * (maxFreq - minFreq) + minFreq)
    cnAmps.append(i/maxVal)
for i in ruData:
    ruFreqs.append(((i - minVal) / (maxVal - minVal)) * (maxFreq - minFreq) + minFreq)
    ruAmps.append(i/maxVal)
for i in usData:
    usFreqs.append(((i - minVal) / (maxVal - minVal)) * (maxFreq - minFreq) + minFreq)
    usAmps.append(i/maxVal)

In [97]:
scDir = '/Applications/SuperCollider/SuperCollider.app/Contents/Resources/'
sc.start( exedir=scDir, verbose=1, spew=0, startscsynth=1, port=57110 )

trying to run scsynth from /Applications/SuperCollider/SuperCollider.app/Contents/Resources/
process.py start_local verbose is 1

... ... Starting Supercollider Server ... ...

/Applications/SuperCollider/SuperCollider.app/Contents/Resources/scsynth -u 57110 -i 2 -o 2
['/done', '/notify']


SynthDef(\HW2, {
	|amp = 0.5, freq = 220, pan = 0|
	var sig = SinOsc.ar(freq, 0, amp);
	Out.ar(0, Pan2.ar(sig, pan, 1.0));
}).store

In [99]:
dur = 20
timeout = time.time() + dur

sCn = sc.Synth( "HW2" )
sRu = sc.Synth( "HW2" )
sUs = sc.Synth( "HW2" )

ampScaler = 0.75
x = 0
while time.time() < timeout:
    time.sleep(1.0/20)
    sCn.freq = cnFreqs[x]
    sCn.amp = ampScaler
    sCn.pan = -1
    sRu.freq = ruFreqs[x]
    sRu.amp = ampScaler
    sRu.pan = 0
    sUs.freq = usFreqs[x]
    sUs.amp = ampScaler
    sUs.pan = 1
    x += 1
    if x >= len(cnFreqs):
        x -= len(cnFreqs)
    
sCn.free()
sRu.free()
sUs.free()

['/n_end', 1001, 0, -1, 1002, 0]
['/n_end', 1002, 0, -1, 1003, 0]
['/n_end', 1003, 0, -1, -1, 0]
1004 < created node id
['/fail', '/n_set', 'Node 1004 not found']
['/fail', '/n_set', 'Node 1004 not found']
['/d_removed', 'HW2']
['/done', '/d_load']
1005 < created node id
1006 < created node id
['/n_go', 1004, 0, -1, -1, 0]
['/n_go', 1005, 0, 1004, -1, 0]
['/n_go', 1006, 0, 1005, -1, 0]
['/fail', '/n_set', 'Node 1004 not found']
['/fail', '/n_set', 'Node 1005 not found']
['/fail', '/n_set', 'Node 1005 not found']
['/fail', '/n_set', 'Node 1006 not found']
['/fail', '/n_set', 'Node 1006 not found']


In [95]:
dur = 20
timeout = time.time() + dur

sCn = sc.Synth( "HW2" )
sRu = sc.Synth( "HW2" )
sUs = sc.Synth( "HW2" )

x = 0
while time.time() < timeout:
    time.sleep(1.0/10)
    sCn.freq = 670
    sCn.amp = cnAmps[x]
    sCn.pan = (cnAmps[x]*2) - 1
    sRu.freq = 432
    sRu.amp = ruAmps[x]
    sRu.pan = (ruAmps[x]*2) - 1
    sUs.freq = 1200
    sUs.amp = usAmps[x]
    sUs.pan = (usAmps[x]*2) - 1
    x += 1
    if x >= len(cnFreqs):
        x -= len(cnFreqs)
    
sCn.free()
sRu.free()
sUs.free()

['/n_end', 1046, 0, -1, 1047, 0]
['/n_end', 1047, 0, -1, 1048, 0]
['/n_end', 1048, 0, -1, -1, 0]
1049 < created node id
['/fail', '/n_set', 'Node 1049 not found']
['/fail', '/n_set', 'Node 1049 not found']
['/d_removed', 'HW2']
['/done', '/d_load']
1050 < created node id
1051 < created node id
['/n_go', 1049, 0, -1, -1, 0]
['/n_go', 1050, 0, 1049, -1, 0]
['/n_go', 1051, 0, 1050, -1, 0]
['/fail', '/n_set', 'Node 1049 not found']
['/fail', '/n_set', 'Node 1050 not found']
['/fail', '/n_set', 'Node 1050 not found']
['/fail', '/n_set', 'Node 1051 not found']
['/fail', '/n_set', 'Node 1051 not found']


KeyboardInterrupt: 

In [93]:
dur = 20
timeout = time.time() + dur

sCn = sc.Synth( "HW2" )
sRu = sc.Synth( "HW2" )
sUs = sc.Synth( "HW2" )

x = 0
while time.time() < timeout:
    time.sleep(1.0/20)
    sCn.freq = cnFreqs[x]
    sCn.amp = cnAmps[x]
    sCn.pan = (cnAmps[x]*2) - 1
    sRu.freq = ruFreqs[x]
    sRu.amp = ruAmps[x]
    sRu.pan = (ruAmps[x]*2) - 1
    sUs.freq = usFreqs[x]
    sUs.amp = usAmps[x]
    sUs.pan = (usAmps[x]*2) - 1
    x += 1
    if x >= len(cnFreqs):
        x -= len(cnFreqs)
    
sCn.free()
sRu.free()
sUs.free()

['/n_end', 1041, 0, -1, 1042, 0]
['/n_end', 1042, 0, -1, -1, 0]
1043['/fail', '/n_set', 'Node 1043 not found']
 < created node id
['/fail', '/n_set', 'Node 1043 not found']
['/d_removed', 'HW2']
['/done', '/d_load']
1044 < created node id
1045 < created node id
['/n_go', 1043, 0, -1, -1, 0]
['/n_go', 1044, 0, 1043, -1, 0]
['/n_go', 1045, 0, 1044, -1, 0]
['/fail', '/n_set', 'Node 1043 not found']
['/fail', '/n_set', 'Node 1044 not found']
['/fail', '/n_set', 'Node 1044 not found']
['/fail', '/n_set', 'Node 1045 not found']
['/fail', '/n_set', 'Node 1045 not found']
['/n_end', 1043, 0, -1, 1044, 0]


In [30]:
sCn.free()
sRu.free()
sUs.free()

['/n_end', 1004, 0, -1, 1005, 0]
['/n_end', 1005, 0, -1, 1006, 0]
['/n_end', 1006, 0, -1, -1, 0]
['/fail', '/n_free', 'Node 1004 not found']
['/fail', '/n_free', 'Node 1005 not found']
['/fail', '/n_free', 'Node 1006 not found']


In [96]:
sc.quit()